In [153]:
# Note: need to run the following in powershell
#       pip install polygon-api-client

import pandas as pd
from polygon import RESTClient

# Note: the furthest back we can go with the free plan seems to be Feb 12, 2023
key = 'API KEY HERE'
client = RESTClient(key)

In [148]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing all info on all stocks at a single specified date
    Date is a string of the form YYYY-MM-DD
    """
    
    data = client.get_grouped_daily_aggs(date)

    df = pd.DataFrame({
        'Close' : [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open' : [stock.open for stock in data],
        'High' : [stock.high for stock in data],
        'Low' : [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
        index = [stock.ticker for stock in data])
    
    df = df.sort_values('Volume', ascending = False)
    
    return df


In [152]:
df = multiple_stock_table('2023-02-13')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
NVDA,21.7880,474918620.0,21.5380,22.0480,20.9620,21.6940,463567.0
SRNE,0.2572,297283013.0,0.6053,0.6600,0.2388,0.3656,227220.0
TSLA,194.6400,172475452.0,194.4150,196.3000,187.6100,193.2998,1522129.0
TQQQ,24.9500,139869852.0,24.1400,25.1000,23.8500,24.6580,296346.0
PLTR,7.6100,87397065.0,7.5800,7.7750,7.4500,8.1066,204947.0
...,...,...,...,...,...,...,...
RNWZ,24.2266,0.0,24.2266,24.2266,24.2266,NaN,NaN
MNBD,25.7430,0.0,25.7430,25.7430,25.7430,NaN,NaN
ESCR,18.2850,0.0,18.2850,18.2850,18.2850,NaN,NaN
CSH,25.4300,0.0,25.4300,25.4300,25.4300,NaN,NaN


In [139]:
from datetime import datetime

# Note:  the API lists timestamps rather than dates
#        these are unix millisecond timestamps, which describe the number of milliseconds 
#        since Jan 1, 1970, midnight UTC/GMT.
#        We use the following function to convert timestamps to date strings

def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    
    # Convert to seconds and shift by a day
    epoch_secs = timestamp/1000 + 86400
    
    # Convert the timestamp to a datetime object
    dt_object = datetime.fromtimestamp(epoch_secs)
    
    # Convert to string of form YYYY-MM-DD
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

In [134]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all info on a given stock from start_date to end_date
    Dates are strings of the form YYYY-MM-DD
    """
    
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    df = pd.DataFrame({
    'Close' : [entry.close for entry in data],
    'Volume': [entry.volume for entry in data],
    'Open' : [entry.open for entry in data],
    'High' : [entry.high for entry in data],
    'Low' : [entry.low for entry in data], 
    'Vwap': [entry.vwap for entry in data],
    'Transactions': [entry.transactions for entry in data]},
    index = [timestamp_to_date(entry.timestamp) for entry in data]
    )
    
    return df



In [143]:
df = single_stock_table('NVDA', '2022-08-01', '2025-02-09')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
2023-02-13,21.788,474918620.0,21.538,22.0480,20.9620,21.6940,463567
2023-02-14,22.971,675435900.0,21.578,23.0488,21.3660,22.5682,697353
2023-02-15,22.764,421148660.0,22.549,22.8550,22.1060,22.5468,471919
2023-02-16,22.002,412026280.0,22.133,22.5500,21.9268,22.2462,473577
2023-02-17,21.388,465887640.0,21.631,21.7400,20.9750,21.3416,552526
...,...,...,...,...,...,...,...
2025-02-03,116.660,366403753.0,114.750,118.5700,113.0100,116.0887,3399816
2025-02-04,118.650,242321420.0,116.960,121.2000,116.7000,119.0311,1841195
2025-02-05,124.830,261217827.0,121.760,125.0000,120.7600,123.0659,1968080
2025-02-06,128.680,248146032.0,127.420,128.7700,125.2100,127.3319,1855949


In [146]:
data = client.get_aggs('NVDA', 1, 'day','2021-08-01', '2025-02-09')
data

[Agg(open=21.538, high=22.048, low=20.962, close=21.788, volume=474918620.0, vwap=21.694, timestamp=1676264400000, transactions=463567, otc=None),
 Agg(open=21.578, high=23.0488, low=21.366, close=22.971, volume=675435900.0, vwap=22.5682, timestamp=1676350800000, transactions=697353, otc=None),
 Agg(open=22.549, high=22.855, low=22.106, close=22.764, volume=421148660.0, vwap=22.5468, timestamp=1676437200000, transactions=471919, otc=None),
 Agg(open=22.133, high=22.55, low=21.9268, close=22.002, volume=412026280.0, vwap=22.2462, timestamp=1676523600000, transactions=473577, otc=None),
 Agg(open=21.631, high=21.74, low=20.975, close=21.388, volume=465887640.0, vwap=21.3416, timestamp=1676610000000, transactions=552526, otc=None),
 Agg(open=21, high=21.4938, low=20.618, close=20.655, volume=410014550.0, vwap=20.9405, timestamp=1676955600000, transactions=427159, otc=None),
 Agg(open=20.707, high=21.104, low=20.421, close=20.754, volume=513088320.0, vwap=21.0021, timestamp=1677042000000, 